# Imports

In [19]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
!export PYSPARK_PYTHON=python
#!apt install default-jre
#!apt install default-jdk+
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
18 packages can be upgraded. Run 'apt list --upgradable' to see them.
openjdk-8-jdk-headless is already the newest version (8u382-ga-1~22.04.1).
0 upgraded, 0 newly ins

In [20]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import math
from datetime import datetime

import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [21]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [22]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Inicialización de los RDD

In [23]:
id1="17SvnFyBNyY018rxrRnOLGp9YaZfeRknp"
downloaded1 = drive.CreateFile({'id': id1})
downloaded1.GetContentFile('googleplaystore.csv')

In [24]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

sqlContext = SQLContext(sc)

apps_df = sqlContext.read.csv('googleplaystore.csv',
                              header=True,
                              inferSchema=True) \
                              .select('App', 'Rating', 'Genres', 'Android Ver')
apps_rdd = apps_df.rdd

# Pregunta #12
[*ir al notebook general*](https://colab.research.google.com/drive/1wWj79hKJTtaauQYJndtlhfQ8KEiuKe8Q?ouid=101857797731019568886&usp=drive_link)

> *Juan tiene un dispositivo un poco viejo con Android 3.9 y ya no lo puede actualizar. Quiere descargar apps del género de “Arte y Diseño” pero no sabe cuáles son válidas para su versión de Android. Ayúdalo diciéndole cuántas apps puede descargarse y mencionale 3 de ellas con su respectivo Rating*

**Respuesta:** se puede descargar 11 apps, ver la salida para las 3 apps mencionadas.

**Notas:**
- Solo me interesaban en verdad los 2 primeros números de la versión de android , por lo que convertirlo a float me bastaba
- Btw, en pandas me fijé cuáles eran las posibles versiones (justo tenía uno de los ej con eso) y ví que el casteo que estoy haciendo no me rompe nada (Los únicos NaN de hecho son los "Varies with device", y todas las versiones que son rangos ni pasan por la 3.9)
- Corrí un par de veces el bloque con `%%timeit` y con el cache funciona *un poquito* más rápido
- El primer map y reduceByKey me quitan las apps duplicadas. Hago básicamente lo mismo (map a dupla nombre - campos que quiero y reduceByKey de agarrar uno de los 2) en el resto de los ejercicios que usan el dataset de apps. No tengo ningún criterio para seleccionar a uno de los duplicados, así que agarro uno cualquiera

In [25]:
def ver_to_float(str):
  try:
    return float(str[:3])
  except:
    return math.nan

ans_rdd = apps_rdd \
  .map(lambda x: (x[0], (x[2], x[3], x[1]))) \
  .reduceByKey(lambda x,y: x) \
  .map(lambda x: (x[0], x[1][0], ver_to_float(x[1][1]), x[1][2])) \
  .filter(lambda x: x[2] <= 3.9 and "Art & Design" in x[1]) \
  .cache()

total = ans_rdd.count()
apps = ans_rdd.take(3)

# Salida
print(f"{total} apps encontradas, entre ellas:")

red = '\033[91m'
yellow = '\033[93m'
white = '\033[97m'

for app in apps:
  print(red+"================================")
  print(f"{yellow}App: {white}{app[0]}")
  print(f"{yellow}Genres: {white}{app[1]}")
  print(f"{yellow}Android ver.: {white}{app[2]}")
  print(f"{yellow}Rating: {white}{app[3]}")
  print(red+"================================"+white)


11 apps encontradas, entre ellas:
App: Paper flowers instructions
Genres: Art & Design
Android ver.: 2.3
Rating: 4.4
App: Garden Coloring Book
Genres: Art & Design
Android ver.: 3.0
Rating: 4.4
App: 3D Color Pixel by Number - Sandbox Art Coloring
Genres: Art & Design
Android ver.: 2.3
Rating: 4.4
